<a href="https://colab.research.google.com/github/aquibjaved/BitsAndPieces-Computation/blob/main/long_seq_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [1]:
import torch
import torch.nn as nn
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizerFast
import datasets
from torch.utils.data import Dataset, DataLoader

from torch.optim import AdamW
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
from transformers import DistilBertModel

In [2]:
# Load the configuration for DistilBERT
config = DistilBertConfig.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
batch_size = 2
seq_len = 700  # Input can be longer than max_seq_len (512)
vocab_size = 30522

# Generate valid input IDs
input_ids = torch.randint(0, vocab_size, (batch_size, seq_len)).to('cuda')

# Forward pass
outputs, hidden_states = model(input_ids, return_hidden_states=True)
print(hidden_states.shape)  # Expected: (batch_size, 512, 768)
print(outputs.shape)  # Expected: (batch_size, num_classes
print(outputs)

torch.Size([2, 512, 768])
torch.Size([2, 2])
tensor([[-0.1684,  0.1270],
        [-0.2044, -0.1317]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [6]:
print("Trainable Parameters and Layers:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name} | Size: {param.size()} | Requires Grad: {param.requires_grad}")

Trainable Parameters and Layers:
Layer: embeddings.word_embeddings.weight | Size: torch.Size([30522, 768]) | Requires Grad: True
Layer: embeddings.position_embeddings.weight | Size: torch.Size([512, 768]) | Requires Grad: True
Layer: embeddings.LayerNorm.weight | Size: torch.Size([768]) | Requires Grad: True
Layer: embeddings.LayerNorm.bias | Size: torch.Size([768]) | Requires Grad: True
Layer: embedding.weight | Size: torch.Size([30522, 768]) | Requires Grad: True
Layer: linear_reduce.weight | Size: torch.Size([768, 768]) | Requires Grad: True
Layer: linear_reduce.bias | Size: torch.Size([768]) | Requires Grad: True
Layer: attention.weight | Size: torch.Size([1, 768]) | Requires Grad: True
Layer: attention.bias | Size: torch.Size([1]) | Requires Grad: True
Layer: classifier.weight | Size: torch.Size([2, 768]) | Requires Grad: True
Layer: classifier.bias | Size: torch.Size([2]) | Requires Grad: True


In [7]:
# Function to calculate and print parameter statistics
def print_trainable_params_percentage(model):
    total_params = sum(p.numel() for p in model.parameters())  # Total parameters
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)  # Trainable parameters

    # Calculate percentage
    percentage = (trainable_params / total_params) * 100 if total_params > 0 else 0

    # Print statistics
    print(f"Total Parameters: {total_params}")
    print(f"Trainable Parameters: {trainable_params}")
    print(f"Percentage of Trainable Parameters: {percentage:.2f}%")


In [8]:
print_trainable_params_percentage(model)

Total Parameters: 90396675
Trainable Parameters: 47869443
Percentage of Trainable Parameters: 52.95%


In [4]:
imdb = datasets.load_dataset("imdb")
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [5]:
from datasets import load_dataset
from transformers import DistilBertTokenizerFast
import matplotlib.pyplot as plt

# Load IMDB dataset
imdb = load_dataset("imdb")

# Load DistilBERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize the training dataset and calculate sequence lengths
train_texts = imdb['train']['text']
tokenized_lengths = [len(tokenizer.encode(text, truncation=False, add_special_tokens=True)) for text in train_texts]

# Plot the distribution of tokenized lengths
plt.figure(figsize=(10, 6))
plt.hist(tokenized_lengths, bins=50, color='skyblue', edgecolor='black')
plt.title("Distribution of Tokenized Sequence Lengths in IMDB Training Dataset", fontsize=16)
plt.xlabel("Number of Tokens", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.grid(alpha=0.3)
plt.show()

# Print basic statistics
print("Basic Statistics:")
print(f"Minimum length: {min(tokenized_lengths)} tokens")
print(f"Maximum length: {max(tokenized_lengths)} tokens")
print(f"Average length: {sum(tokenized_lengths) / len(tokenized_lengths):.2f} tokens")


Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [6]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


In [7]:
train_texts = imdb['train']['text']
train_labels = imdb['train']['label']

test_texts = imdb['test']['text']
test_labels = imdb['test']['label']


max_len = 700
batch_size = 64

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

train_dataset = TextDataset(train_texts, train_labels, tokenizer, max_len)
test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_len)


train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)


In [11]:
from tqdm.notebook import tqdm

In [12]:
from IPython.display import clear_output
from tqdm import tqdm
from IPython.display import clear_output

def train(model, dataloader, optimizer, criterion, device, num_epochs):
    """
    Train the model for a specified number of epochs.

    Args:
        model (nn.Module): The model to train.
        dataloader (DataLoader): The DataLoader for training data.
        optimizer (Optimizer): The optimizer for gradient descent.
        criterion (Loss): The loss function.
        device (torch.device): The device (CPU or GPU) for training.
        num_epochs (int): The number of epochs to train.

    Returns:
        None
    """
    model.train()

    for epoch in range(num_epochs):
        print(f"Epoch [{epoch + 1}/{num_epochs}] starting...")
        total_loss = 0
        correct_predictions = 0
        total_samples = 0

        # Use tqdm to wrap the dataloader for a progress bar
        progress_bar = tqdm(dataloader, desc=f"Epoch [{epoch + 1}/{num_epochs}]")

        for batch_idx, batch in enumerate(progress_bar):
            optimizer.zero_grad()

            # Move data to the device
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)

            # Forward pass
            outputs = model(input_ids)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Calculate accuracy for this batch
            _, preds = torch.max(outputs, dim=1)
            correct_predictions += (preds == labels).sum().item()
            total_samples += labels.size(0)

            # Update progress bar description
            progress_bar.set_postfix(
                Loss=f"{loss.item():.4f}",
                AvgLoss=f"{total_loss / (batch_idx + 1):.4f}"
            )

        # Calculate and log epoch accuracy
        accuracy = correct_predictions / total_samples
        avg_loss = total_loss / len(dataloader)
        clear_output(wait=True)
        print(f"Epoch [{epoch + 1}/{num_epochs}] completed. Average Loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}\n")

    print("Training complete.")



def evaluate(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            # Move data to the device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs, dim=1)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predictions)
    return accuracy


In [9]:
class MergedDistilBertModel(DistilBertModel):
    def __init__(self, config, vocab_size=30522, embed_dim=768, max_seq_len=512, padding_idx=0, num_classes=2):
        super().__init__(config)

        self.max_seq_len = max_seq_len

        # Layers from SequenceReducer
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
        self.linear_reduce = nn.Linear(embed_dim, embed_dim)
        self.attention = nn.Linear(embed_dim, 1)
        self.pooling = nn.AdaptiveAvgPool2d((max_seq_len, embed_dim))

        # Freeze all DistilBERT parameters
        for param in self.transformer.parameters():  # Only freeze transformer weights
            param.requires_grad = False

        # Classification head
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, input_ids, attention_mask=None, return_hidden_states=False, **kwargs):
        """
        Forward pass combining SequenceReducer and CustomDistilBertModel.

        Args:
            input_ids (torch.Tensor): Token IDs of shape (batch_size, seq_len).
            attention_mask (torch.Tensor): Optional attention mask of shape (batch_size, seq_len).
            return_hidden_states (bool): Whether to return hidden states.

        Returns:
            torch.Tensor: Logits for classification of shape (batch_size, num_classes).
            Optional[torch.Tensor]: Hidden states of shape (batch_size, max_seq_len, embed_dim) if `return_hidden_states` is True.
        """
        # SequenceReducer functionality
        embeddings = self.embedding(input_ids)  # Shape: (batch_size, seq_len, embed_dim)
        attention_scores = self.attention(embeddings)  # Shape: (batch_size, seq_len, 1)
        attention_weights = torch.softmax(attention_scores, dim=1)  # Normalize scores
        attended_embeddings = embeddings * attention_weights  # Shape: (batch_size, seq_len, embed_dim)
        reduced_embeddings = self.linear_reduce(attended_embeddings)  # Shape: (batch_size, seq_len, embed_dim)
        pooled_embeddings = self.pooling(reduced_embeddings.transpose(1, 2))  # Shape: (batch_size, max_seq_len, embed_dim)

        # Pass reduced embeddings through DistilBERT (CustomDistilBertModel functionality)
        position_ids = torch.arange(pooled_embeddings.size(1), dtype=torch.long, device=pooled_embeddings.device).unsqueeze(0)
        position_embeddings = self.embeddings.position_embeddings(position_ids)

        # Apply position embeddings and normalization/dropout layers
        embeddings = pooled_embeddings + position_embeddings
        embeddings = self.embeddings.LayerNorm(embeddings)
        embeddings = self.embeddings.dropout(embeddings)

        # print("Input Embeddings shape: ", embeddings.shape)
        # print("attention_mask shape: ", attention_mask.shape)

        # DistilBERT forward pass
        hidden_states = super().forward(inputs_embeds=embeddings, attention_mask=attention_mask, **kwargs).last_hidden_state

        # Classification head: Take [CLS]-like token (first token) for classification
        cls_token = hidden_states[:, 0, :]  # Shape: (batch_size, embed_dim)
        logits = self.classifier(cls_token)  # Shape: (batch_size, num_classes)

        if return_hidden_states:
            return logits, hidden_states
        return logits


In [ ]:
model = MergedDistilBertModel(config).to('cuda')

device = "cuda" if torch.cuda.is_available() else "cpu"
num_epochs = 3

# Define optimizer and loss
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()
# def train(model, dataloader, optimizer, criterion, device):
train(model, train_dataloader, optimizer, criterion, device, num_epochs)

Epoch [2/3] completed. Average Loss = 0.7052, Accuracy = 0.5051

Epoch [3/3] starting...


Epoch [3/3]:   1%|          | 3/391 [00:07<16:54,  2.62s/it, AvgLoss=0.6933, Loss=0.7142]